<h1>Assignment 3</h1>

<h3> <a href="https://colab.research.google.com/drive/1j4I1iVzW0DlXZQmpxlkhUHBlAfnquu7C?usp=sharing">Assignment 3 Colab Link</a></h3>

In [233]:
import pandas as pd
import numpy as np


ZIP_FILE_LINK = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'

<h4>Loading data</h4>

In [234]:
# Fetch data from ZIP file link and store converted CSV
def fetch_data():
    import requests, zipfile, io
    r = requests.get(ZIP_FILE_LINK)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall()

# Check if the folder already exists
try:
    open('ml-latest-small/ratings.csv')
except FileNotFoundError:
    print('Fetching data...')
    fetch_data()

# Read CSV files
ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
links = pd.read_csv('ml-latest-small/links.csv')
tags = pd.read_csv('ml-latest-small/tags.csv')


In [235]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


<h2>Part A</h2>

<p>Motivated by the sequential methods we discussed in class (please see also at the
corresponding research paper https://homepages.tuni. /konstantinos.stefanidis/docs/
sac20.pdf), the goal of the third assignment is to design (Score: 30%) and implement
(Score: 30%) a new method for producing sequential group recommendations. Also,
provide detailed explanations and clarifications about why the method you propose works
well for the case of sequential group recommendations (Score: 25%).</p>


<p>First let's get helper functions for predicting movie ratings from the previous assignment and prepare data</p>

In [236]:
# Helper functions from the previous assignment

# min_common_percentage = 0.1
min_common_items = 2

# Implementation of Pearson correlation
def get_similarity_between_two_items(user1, user2):
    # Find common items
    common_items = user1.notna() & user2.notna()
    
    common_items_count = common_items.sum()
    if common_items_count == 0:
        return 0  # No common items, no correlation
    
    # We implement a treshold to avoid meaningless correlations
    # Approach 1: Common items percentage
    # total_items_user1 = user1.count()
    # total_items_user2 = user2.count()
    # common_percentage_user1 = common_items_count / total_items_user1
    # common_percentage_user2 = common_items_count / total_items_user2
    # if common_percentage_user1 < min_common_percentage or common_percentage_user2 < min_common_percentage:
    #     return 0  # Not enough common items for a meaningful correlation

    # Approach 2: Common items count
    if common_items_count < min_common_items:
        return 0  # Not enough common items for a meaningful correlation
    
    # Get the common items
    user1_common = user1[common_items]
    user2_common = user2[common_items]
    
    # Pearson correlation requires at least 2 common items
    if len(user1_common) < 2:
        return 0 
    
    # Calculate the Pearson correlation coefficient
    correlation = user1_common.corr(user2_common)
    
    if np.isnan(correlation):
        return 0  # Handle NaN values
    
    return max(correlation, 0)  # Return a non-negative correlation

def predict_rating(user_id, movie_id, ratings_by_users):
    # If the user has already rated the movie, return the known rating
    if not np.isnan(ratings_by_users.loc[user_id, movie_id]):
        return ratings_by_users.loc[user_id, movie_id]

    # Get the users who rated the movie
    users_who_rated = ratings_by_users[ratings_by_users[movie_id].notna()].index

    # Calculate the similarities and the weighted ratings
    similarities = [get_similarity_between_two_items(ratings_by_users.loc[user_id], ratings_by_users.loc[other_user_id]) for other_user_id in users_who_rated]
    weighted_ratings = [similarity * (ratings_by_users.loc[other_user_id, movie_id] - ratings_by_users.loc[other_user_id].mean()) for other_user_id, similarity in zip(users_who_rated, similarities)]

    # If no one else rated the movie, return the mean rating of the user
    if sum(similarities) == 0:
        return ratings_by_users.loc[user_id].mean()

    # Return the weighted average rating, ensuring it is within the range of 0 - 5
    return max(min((sum(weighted_ratings) / sum(similarities)) + ratings_by_users.loc[user_id].mean(), 5), 0)


In [237]:
# Copying the ratings dataframe to a new dataframe for further processing
movie_ratings = ratings.copy()

In [238]:
# Making a pivot table to get the ratings of each movie by each user
ratings_by_users = movie_ratings.pivot_table(index='userId', columns='movieId', values='rating', aggfunc='first')
ratings_by_users.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h4>Designing and implementing a new method for producing sequential group recommendations<h4>

<p>First we added 2 new group rating prediction strategies: "hybrid" and "fairness"</p>

<h5> Hybrid strategy explanation <h5>

This "hybrid" strategy combines the "average" and "least misery" strategies using a weighted average, providing a balance between the two approaches. The "average" strategy calculates the mean rating for each movie across all users in the group, which can provide a broad appeal as it considers the general consensus of the group. However, it may overlook individual preferences, especially for users whose tastes significantly differ from the group average.

On the other hand, the "least misery" strategy selects the minimum rating for each movie across all users, aiming to avoid any potential dissatisfaction within the group. This strategy ensures that the recommended items are at least tolerable to all users. However, it can be overly conservative, potentially missing out on items that are highly rated by a subset of the group. By combining these two strategies, the "hybrid" approach can provide recommendations that both appeal to the group as a whole and avoid causing strong dissatisfaction among any of its members.

<h5>Fairness stragegy explanation</h5>

The "fairness" strategy in this context is designed to balance the satisfaction of all users in a group when generating recommendations. The first step in this strategy is to calculate the average rating for each movie across all users in the group. This average rating represents the general consensus of the group about each movie.

The second step is to adjust this average rating by the variance of the ratings. The variance measures how much the ratings for each movie differ among the users in the group. By subtracting this variance from the average rating, the strategy gives higher scores to movies that are rated more consistently by the group. This means that movies with similar ratings from all users are preferred, promoting fairness in the group recommendation. This strategy can be particularly useful in situations where it's important to avoid favoring some users over others.

In [239]:
# The function has some extensive parameters to allow for more flexibility
# userIds: The ids of the users we want to predict the ratings for
# all_users_ratings_copy: The movie ratings by all users
# strategy: The strategy to use for the prediction. Can be 'average' or 'least_misery'

def group_rating_prediction(userIds, all_users_ratings, strategy='average', useFullDataset=False, hybrid_weight=None):
    # Get only the data about the users we are interested in
    group_users_ratings = all_users_ratings.loc[userIds]

    # Remove the movies that no one has rated
    group_users_ratings = group_users_ratings.dropna(axis=1, how='all')

    # Dataset to use for prediction
    if(useFullDataset):
        dataset_for_prediction = all_users_ratings.copy()
    else:
        dataset_for_prediction = group_users_ratings.copy()

    # Predict the individual ratings of the movies that the users haven't rated
    for user_id in userIds:
        for movie_id in group_users_ratings.columns:
            group_users_ratings.loc[user_id, movie_id] = predict_rating(user_id, movie_id, dataset_for_prediction)
    
    if(strategy == 'average'):
        # Get the average rating for every movie
        movie_ratings_average = group_users_ratings.mean(axis=0)

    if(strategy == 'least_misery'):
        # Get the least misery rating for every movie
        movie_ratings_average = group_users_ratings.min(axis=0)

    if(strategy == 'fairness'):
        # Fairness strategy 
        # Get the average rating for every movie
        movie_ratings_average = group_users_ratings.mean(axis=0)
        # Adjust the average rating by the variance of ratings
        movie_ratings_variance = group_users_ratings.var(axis=0)
        movie_ratings_average -= movie_ratings_variance

    if(strategy == 'hybrid' and hybrid_weight is not None): 
        # Get the average rating for every movie
        movie_ratings_average_average = group_users_ratings.mean(axis=0)
        # Get the least misery rating for every movie, ignore NaN values
        movie_ratings_average_least_misery = group_users_ratings.min(axis=0)
        # Calculate the hybrid rating
        movie_ratings_average = (1 - hybrid_weight) * movie_ratings_average_average + hybrid_weight * movie_ratings_average_least_misery

    # Sort the movies by their average rating
    movie_ratings_average = movie_ratings_average.sort_values(ascending=False)

    return movie_ratings_average


<p>Secondly we need to implement a sequence prediction method<p>

The function `sequential_group_recommendation` is designed to generate a sequence of movie recommendations for a group of users. It takes as input a list of user IDs, a DataFrame of all users' ratings, and optional parameters to control the number of recommendations and iterations, and whether to use an explicit recommendation strategy.

The function operates in a loop for a specified number of iterations. In each iteration, it calculates a "satisfaction difference" based on the maximum and minimum user satisfaction scores from the previous iteration. It then chooses a recommendation strategy based on this satisfaction difference or an explicitly provided strategy. Using the chosen strategy, it generates a group recommendation and calculates the satisfaction of each user with this recommendation. It also calculates the average group satisfaction for this iteration and the overall sequence satisfaction. The recommended movies are then removed from the DataFrame of ratings, and the process repeats for the next iteration. The function returns the sequence of all recommendations, the satisfaction scores of each user at each iteration, the group satisfaction scores for each iteration, and the overall sequence satisfaction score.

In [240]:
# Satisfaction difference thresholds for choosing the strategy
high_threshold = 0.75
low_threshold = 0.10

from scipy.stats import pearsonr

def calculate_group_similarity(userIds, all_users_ratings):
    # Extract the ratings of the users in the group
    group_ratings = all_users_ratings.loc[userIds]

    # Calculate the pairwise Pearson correlation between all users in the group
    similarity_scores = []
    for i in range(len(userIds)):
        for j in range(i+1, len(userIds)):
            user1_ratings = group_ratings.loc[userIds[i]].dropna()
            user2_ratings = group_ratings.loc[userIds[j]].dropna()
            common_ratings = user1_ratings.index.intersection(user2_ratings.index)
            if len(common_ratings) >= 2:
                similarity, _ = pearsonr(user1_ratings[common_ratings], user2_ratings[common_ratings])
                similarity_scores.append(similarity)
            else:
                similarity_scores.append(0)

    # Average the pairwise similarity scores to get a single similarity score for the group
    group_similarity = np.mean(similarity_scores)

    # Should be in range 0 - 1 
    group_similarity =  (group_similarity + 1) / 2

    return group_similarity

def calculate_user_satisfaction(userId, all_users_ratings_copy, group_recommendation, numberOfRecommendations=10): 
    # Top N ratings the user has given
    user_ratings = all_users_ratings_copy.loc[userId].sort_values(ascending=False).head(numberOfRecommendations)

    # Convert the index of group_recommendation to a list of integers
    group_recommendation_index = group_recommendation.head(numberOfRecommendations).index.tolist()

    # Sum the scores by getting IDs from group recommendation and using ratings dataframe
    group_list_sat = all_users_ratings_copy.loc[userId, group_recommendation_index].sum()

    user_list_sat = user_ratings.sum()

    # Calculate the user satisfaction
    return group_list_sat / user_list_sat


def sequential_group_recommendation(userIds, all_users_ratings, useFullDataset=True, numberOfRecommendations=10, numberOfIterations=10, explicit_strategy=None):
    # Get the copy of the ratings dataframe
    all_users_ratings_copy = all_users_ratings.copy()

    # Final recommendations
    all_recommendations = []
    # User satisfactions
    user_satisfactions = pd.DataFrame(index=userIds, columns=range(numberOfIterations))
    # Group recommendation satisfaction
    group_recommendation_satisfaction = []

    # Calculate the group similarity for the first iteration
    group_similarity = calculate_group_similarity(userIds, all_users_ratings_copy)

    for i in range(numberOfIterations):
        # Get satisfaction from previous iteration
        previous_satisfaction = user_satisfactions[i - 1] if i > 0 else None

        # Calculate the hybrid weight and satisfaction difference
        if previous_satisfaction is not None:
            satisfaction_difference = previous_satisfaction.max() - previous_satisfaction.min()
        else:
            # If there is no previous satisfaction, use the group similarity
            # The more similar the group is the more we want to use the average strategy for the first iteration
            satisfaction_difference = 1 - group_similarity

        # Choose the strategy based on the explicit_strategy parameter or satisfaction difference
        if explicit_strategy is not None:
            strategy = explicit_strategy
        elif satisfaction_difference is not None:
            if satisfaction_difference > high_threshold:
                strategy = 'fairness'
            elif satisfaction_difference < low_threshold:
                strategy = 'average'
            else:
                strategy = 'hybrid'
        else:
            strategy = 'average'

        # Get the group recommendation using the chosen strategy
        group_recommendation = group_rating_prediction(userIds, all_users_ratings_copy, useFullDataset=useFullDataset, hybrid_weight=satisfaction_difference, strategy=strategy)

        # Get the top 10 recommendations and add them
        recommendations = group_recommendation.head(numberOfRecommendations).index.tolist()

        # Add the recommendations to the final recommendations
        all_recommendations.append(recommendations)

        # Calculate the user satisfaction using the calculate_user_satisfaction function
        for userId in userIds:
            user_satisfactions.loc[userId, i] = calculate_user_satisfaction(userId, all_users_ratings_copy, group_recommendation, numberOfRecommendations=numberOfRecommendations)

        # Store the group satisfaction for this iteration
        group_satisfaction = user_satisfactions[i].mean()
        group_recommendation_satisfaction.append(group_satisfaction)

        # Calculate the recommendation sequence satisfaction
        sequence_satisfaction = np.mean(group_recommendation_satisfaction)
        # Remove recommended movies from the copy of the ratings dataframe
        all_users_ratings_copy = all_users_ratings_copy.drop(group_recommendation.head(numberOfRecommendations).index.tolist(), axis=1, errors='ignore')

    return all_recommendations, user_satisfactions, group_recommendation_satisfaction, sequence_satisfaction

<h5>Explanation and clarification on the chosen method</h5>

The main advantage of the chosen method is dynamic strategy selection and possibility to configure tresholds for it. It allows the recommendation system to adapt to the group's satisfaction level. This is done by calculating the satisfaction difference between the most and least satisfied users in the previous iteration, and then choosing a recommendation strategy based on this difference.

If the satisfaction difference is high, the 'fairness' strategy is chosen to balance the satisfaction levels of all users. This strategy is designed to avoid favoring some users over others, which can be particularly useful when there's a large disparity in satisfaction levels within the group.

If the satisfaction difference is low, the 'average' strategy is chosen to focus on the general consensus of the group. This strategy can be beneficial when the group's satisfaction levels are already balanced, as it aims to maximize the overall group satisfaction.

If the satisfaction difference is moderate, the 'hybrid' strategy is chosen to provide a balance between the 'average' and 'fairness' strategies. This strategy can provide a good compromise when there's a moderate disparity in satisfaction levels within the group.

By dynamically selecting the recommendation strategy based on the group's satisfaction level, the function can adapt to the group's needs and potentially improve the quality of the recommendations. This makes the recommendation system more flexible and responsive, which can lead to higher user satisfaction.

<h5>Produce a group of 3 users, and for this group, show the top-10 recommendations in 3
different sequences, i.e., the 10 movies with the highest prediction scores in 3 rounds,
using the MovieLens 100K rating dataset (Score: 5%). </h5>

In [241]:
# Calculate satisfaction for a given group recommendation
userIds = [3, 8, 12]
numberOfIteration = 3

(
    recommendations,
    user_satisfactions,
    group_satisfaction,
    sequence_satisfaction,
) = sequential_group_recommendation(
    userIds,
    ratings_by_users,
    useFullDataset=True,
    numberOfRecommendations=10,
    numberOfIterations=numberOfIteration,
)


/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


Recommended movies per iteration

In [242]:
recommendations

[[3703, 2288, 296, 5746, 5181, 6835, 5919, 7899, 5764, 1587],
 [1275, 4518, 849, 150, 2851, 1265, 1371, 3024, 593, 47],
 [26409, 7991, 380, 50, 3967, 318, 2105, 32, 590, 1357]]

User satisfactions per iteration

In [243]:
user_satisfactions

,0,1,2
3,0.85,0.611765,0.589744
8,0.08,0.24,0.46
12,0.0,0.07,0.19


Overall group satisfaction per iteration

In [244]:
group_satisfaction

[0.31, 0.30725490196078437, 0.4132478632478633]

Sequence satisfaction

In [245]:
sequence_satisfaction

0.3435009217362159

Let's compare with "average" strategy

In [246]:
# Calculate satisfaction for a given group recommendation with explicit strategy
(
    recommendations,
    user_satisfactions,
    group_satisfaction,
    sequence_satisfaction,
) = sequential_group_recommendation(
    userIds,
    ratings_by_users,
    useFullDataset=True,
    numberOfRecommendations=10,
    numberOfIterations=numberOfIteration,
    explicit_strategy='average'
)
sequence_satisfaction

/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


0.34978812585780533

As it can be seen the satisfaction is on the same level, but if we add more iterations the dynamic selection system will show better result because with the decreasing number of movies it will be adapting better. Let's try to compare it with average and least misery

In [247]:
# Calculate satisfaction for a given group recommendation
userIds = [3, 8, 12]
numberOfIteration = 4

(
    dynamic_recommendations,
    dynamic_user_satisfactions,
    dynamic_group_satisfaction,
    dynamic_sequence_satisfaction,
) = sequential_group_recommendation(
    userIds,
    ratings_by_users,
    useFullDataset=True,
    numberOfRecommendations=10,
    numberOfIterations=numberOfIteration,
)

(
    average_recommendations,
    average_user_satisfactions,
    average_group_satisfaction,
    average_sequence_satisfaction,
) = sequential_group_recommendation(
    userIds,
    ratings_by_users,
    useFullDataset=True,
    numberOfRecommendations=10,
    numberOfIterations=numberOfIteration,
    explicit_strategy='average'
)

# Least misery
(
    least_misery_recommendations,
    least_misery_user_satisfactions,
    least_misery_group_satisfaction,
    least_misery_sequence_satisfaction,
) = sequential_group_recommendation(
    userIds,
    ratings_by_users,
    useFullDataset=True,
    numberOfRecommendations=10,
    numberOfIterations=numberOfIteration,
    explicit_strategy='least_misery'
)

/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/dmitrii/.local/lib/python3.10/site-packages/nu

In [248]:
# Print all the satisfaction values
print('Dynamic strategy:', dynamic_sequence_satisfaction, 'Average strategy:', average_sequence_satisfaction, 'Least misery strategy:', least_misery_sequence_satisfaction, sep='\n')

Dynamic strategy:
0.372900290844496
Average strategy:
0.33678248633474595
Least misery strategy:
0.3074074373339079


As it was expected the dynamic strategy is performing better with the increasing number of iterations